### Prompts

In this notebook we will go over the following topics-

1. Prompt Templated
2. Connecting to Feature Store

In [1]:
from langchain import PromptTemplate

In [2]:
template = """
I want you to act as a naming consultant for new companies. What is a good name for a company that makes {product}?
"""

prompt = PromptTemplate(
    input_variables = ["product"],
    template = template,
)

prompt.format(product = "colorful gloves")

'\nI want you to act as a naming consultant for new companies. What is a good name for a company that makes colorful gloves?\n'

In [3]:
template = "Tell me a {adjective} joke about {content}."

prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['adjective', 'content'], output_parser=None, partial_variables={}, template='Tell me a {adjective} joke about {content}.', template_format='f-string', validate_template=True)

In [4]:
prompt_template.format(adjective = "funny", content = "chickens")

'Tell me a funny joke about chickens.'

In [5]:
# Serialize the prompt template -
prompt_template.save("test_prompt.json")

In [6]:
from langchain.prompts import load_prompt
loaded_prompt = load_prompt('test_prompt.json')

In [7]:
assert prompt_template == loaded_prompt

Useful prompts from langchain : https://github.com/hwchase17/langchain-hub/tree/master/prompts

### Pass few shot examples to a prompt template

In [8]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [9]:
# create the list of few shot examples -
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

In [11]:
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""

example_prompt = PromptTemplate(
    input_variables = ["word", "antonym"],
    template = example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = "Give the antonym of every input",
    suffix = "Word: {input}\nAntonym:",
    input_variables = ["input"],
    example_separator = "\n\n",
)

In [12]:
print(few_shot_prompt.format(input = "little"))

Give the antonym of every input


Word: happy
Antonym: sad




Word: tall
Antonym: short



Word: little
Antonym:


### large number of examples -

In [13]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [16]:
example = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]


In [17]:
example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    max_length = 25,
)

In [18]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "Give the antonym for every input",
    suffix = "Word: {input}\nAntonym:",
    input_variables = ["input"],
    example_separator = "\n\n"
)

In [19]:
print(dynamic_prompt.format(input = "big"))

Give the antonym for every input


Word: happy
Antonym: sad




Word: tall
Antonym: short



Word: big
Antonym:


## Connecting to Feature Store
Feature store makes sure that the data fed into models is up-to-date and relevant. 

In order to personalize LLM applications, you may need to combine LLM with up-to-date information about particular users. Langchain facilitates it well. It provides an easy way to combine that data with the LLMs.

Connect prompt template to feature stores

### Feast - It is an opensource feature store. 

GitHub: https://github.com/feast-dev/feast
More information : https://docs.feast.dev/

In [21]:
from feast import FeatureStore
import pandas as pd
from datetime import datetime

In [26]:
entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003, 1004],
    "event_timestamp": [
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8,  12, 10),
        datetime(2021, 4, 12, 16, 40, 26),
        datetime(2021, 4, 12, 15, 1 , 12)
    ]
})

store = FeatureStore(repo_path = "./../my_feature_repo/feature_repo/")

training_df = store.get_historical_features(
    entity_df=entity_df,
    features = [
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
).to_df()

print(training_df.head())

   driver_id           event_timestamp  conv_rate  acc_rate  avg_daily_trips
0       1001 2021-04-12 10:59:42+00:00   0.121119  0.320860              201
1       1002 2021-04-12 08:12:10+00:00   0.197364  0.254644              519
2       1003 2021-04-12 16:40:26+00:00   0.976411  0.282935              351
3       1004 2021-04-12 15:01:12+00:00   0.802593  0.971781              140


In [29]:
### Load feature values into your online store

#CURRENT_TIME=$(date -u +"%Y-%m-%dT%H:%M:%S")
#feast materialize-incremental $CURRENT_TIME

In [30]:
from pprint import pprint
from feast import FeatureStore

In [31]:
store = FeatureStore(repo_path = "./../my_feature_repo/feature_repo/")

In [32]:
# Retrieves the latest online feature data.

feature_vector = store.get_online_features(
    features = [
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
    entity_rows = [{"driver_id": 1001}]
).to_dict()

pprint(feature_vector)

{'acc_rate': [0.2518860697746277],
 'avg_daily_trips': [873],
 'conv_rate': [0.2898779809474945],
 'driver_id': [1001]}


Here we will set up a custom FeatPromptTemplate. This prompt template will take in a driver id, look up their stats, and format those stats into a prompt.

Note that the input to this prompt template is just driver_id, since that is the only user defined piece (all other variables are looked up inside the prompt template.)

In [33]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [34]:
template = """ Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above 0.5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better.

Here are the driver stats:
Conversation rate: {conv_rate}
Acceptance rate: {acc_rate}
Average Daily Trips: {avg_daily_trips}

Your response:"""

prompt = PromptTemplate.from_template(template)

In [43]:
class FeastPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        driver_id = kwargs.pop("driver_id")
        feature_vector = store.get_online_features(
            features = [
                'driver_hourly_stats:conv_rate',
                'driver_hourly_stats:acc_rate',
                'driver_hourly_stats:avg_daily_trips'      
            ],
            entity_rows = [{"driver_id": 1001}]
        ).to_dict()
        kwargs["conv_rate"] = feature_vector["conv_rate"][0]
        kwargs["acc_rate"] = feature_vector["acc_rate"][0]
        kwargs["avg_daily_trips"] = feature_vector["avg_daily_trips"][0]
        return prompt.format(**kwargs)

In [44]:
prompt_template = FeastPromptTemplate(input_variables = ["driver_id"])
print(prompt_template.format(driver_id = 1002))

 Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above 0.5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better.

Here are the driver stats:
Conversation rate: 0.2898779809474945
Acceptance rate: 0.2518860697746277
Average Daily Trips: 873

Your response:


In [45]:
# Use in a chain

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [47]:
chain = LLMChain(llm = ChatOpenAI(), prompt = prompt_template)

In [49]:
chain.run(1001)

"Hello there! Just wanted to give you an update on your stats. Your acceptance rate is currently at 0.2518860697746277 and your average daily trips are going strong at 873. While your conversation rate is at 0.2898779809474945, don't worry, there's always room for improvement. If it makes you feel any better, I heard chickens are great at conversation too! Keep up the good work!"

### Tecton : 
Tecton is a fully managed feature platform built to orchestrate the complete ML feature lifecycle, from transformation to online serving, with enterprise-grade SLAs.

https://www.tecton.ai/

### Custom Prompt Template

There are two kinds of prompt templates available -
1. String prompt template
2. Chat prompt template

#### String Prompt template

Here we will create a custom prompt template that takes in the function name as input and formats the prompt to provide the source code of the function.

In [51]:
import inspect

In [52]:
def get_source_code(function_name):
    # get the source code of the function
    return inspect.getsource(function_name)

In [53]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

In [56]:
class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """ A custom prompt template that takes in the function name as input, and formats the prompt template to provide the source code of the function . """
    
    @validator("input_variables")
    def validate_input_variables(cls, v):
        """ validate that the input variables are correct. """
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input variable.")
        return v
    
    def format(self, **kwargs) -> str:
        # Get the source code of the function 
        source_code = get_source_code(kwargs["function_name"])
        
        # Generate the prompt to be sent to the language model
        prompt = f"""
        Given the function name and source code, generate an English language explanation of the function.
        Function Name: {kwargs["function_name"].__name__}
        Source Code:
        {source_code}
        Explanation:
        """
        return prompt
        
    def _prompt_type(self):
        return "function-explainer"

In [58]:
# Now let's generate a custom prompt for our task

In [59]:
# Generate a prompt for the function "get_source_code"

fn_explainer = FunctionExplainerPromptTemplate(input_variables = ["function_name"])

In [64]:
prompt = fn_explainer.format(function_name = get_source_code)

In [61]:
print(prompt)


        Given the function name and source code, generate an English language explanation of the function.
        Function Name: get_source_code
        Source Code:
        def get_source_code(function_name):
    # get the source code of the function
    return inspect.getsource(function_name)

        Explanation:
        
